#### For each match, for all players, we want: team name, player name, position, rating, minutes played, stats
#### Also get date of match. With this, we can use date of birth to find the age. Also useful for tracking rating before and after certain dates
#### The dataframe could look like:

#### Match date | Season | Team name | Player's name | Player's d.o.b. | Player's position | Player's rating | Minutes played | Other player stats (many columns)

In [8]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options as ChromeOptions
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time

# Initialize the WebDriver with custom options
def initialize_driver():
    chrome_options = ChromeOptions()
    # Set the page load strategy to 'none'
    chrome_options.page_load_strategy = 'none'
    
    return webdriver.Chrome(options=chrome_options)

driver = initialize_driver()

def navigate_to_page(url):
    """Navigate to the given URL without waiting for the full page load."""
    driver.get(url)
    time.sleep(2)  # Short initial wait to start loading the page

def scroll_to_matches():
    """Scroll down to the Matches section of the page."""
    try:
        # Scroll to the estimated position to bring the Matches section into view
        scroll_position = 1700  # Adjusted value for the Matches section
        driver.execute_script(f"window.scrollTo(0, {scroll_position});")
        print(f"Scrolled to position: {scroll_position}")
    except Exception as e:
        print(f"An error occurred while scrolling: {e}")

def collect_match_ids(match_ids, round_number):
    """Collect unique match IDs from the currently displayed Matches section."""
    try:
        # Ensure the elements are fully loaded
        WebDriverWait(driver, 10).until(
            EC.presence_of_all_elements_located((By.CSS_SELECTOR, 'a[href*="/football/match/"]'))
        )
        # Second scroll to ensure all matches are visible
        scroll_to_matches()
        time.sleep(1)  # Allow time for full rendering after scroll

        match_links = driver.find_elements(By.CSS_SELECTOR, 'a[href*="/football/match/"]')
        print(f"Round {round_number}: Match links located (count: {len(match_links)}).")

        if len(match_links) < 10:
            print(f"Warning: Less than 10 matches found for round {round_number}, possible loading issue.")

        for match in match_links:
            match_url = match.get_attribute('href')
            match_id = match_url.split('#id:')[-1]
            if match_id not in match_ids:
                match_ids.append(match_id)
                print(f"Found match: {match_url} with Match ID: {match_id}")
    except Exception as e:
        print(f"Error occurred during match collection: {e}")

def find_and_click_left_arrow():
    """Find and click the left arrow button to navigate to the previous round."""
    try:
        left_arrow = WebDriverWait(driver, 10).until(
            EC.visibility_of_element_located((By.CSS_SELECTOR, "button.Button.iCnTrv[style*='visibility: visible;']"))
        )
        if left_arrow.is_displayed() and left_arrow.is_enabled():
            driver.execute_script("arguments[0].click();", left_arrow)
            print("Clicked on the left arrow button.")
            return True
    except Exception as e:
        print(f"Failed to click the left arrow button: {e}")
    return False

def scrape_premier_league_matches():
    """Main function to scrape match IDs from the Premier League page."""
    match_ids = []
    total_rounds = 38  # Number of rounds in the Premier League
    round_click_count = 0

    # Navigate to the Premier League page for the desired season
    navigate_to_page("https://www.sofascore.com/tournament/football/england/premier-league/17#id:52186")
    scroll_to_matches()

    # Collect match IDs for the initial round
    collect_match_ids(match_ids, round_click_count + 1)

    while round_click_count < (total_rounds - 1):  # Loop for 37 left arrow clicks
        try:
            # Attempt to click the left arrow button
            if not find_and_click_left_arrow():
                print("Left arrow button is no longer interactable. Ending loop.")
                break

            round_click_count += 1
            print(f"Total round navigations (left arrow clicks): {round_click_count}")

            # Wait briefly to allow the page to load partially
            time.sleep(3)  # Wait time increased to 3 seconds to allow more content to load

            # Collect match IDs for the current round
            collect_match_ids(match_ids, round_click_count + 1)
            
        except Exception as e:
            print(f"An error occurred during scraping: {e}")
            break

    return match_ids, round_click_count

# Example usage
collected_match_ids, round_click_count = scrape_premier_league_matches()
print("Collected Unique Match IDs:", collected_match_ids)
print(f"Total round navigations (left arrow clicks): {round_click_count}")

# Close the browser after scraping
driver.quit()

Scrolled to position: 1600
Scrolled to position: 1600
Round 1: Match links located (count: 12).
Found match: https://www.sofascore.com/football/match/west-ham-united-manchester-city/rM#id:11352571 with Match ID: 11352571
Found match: https://www.sofascore.com/football/match/everton-arsenal/RY#id:11352546 with Match ID: 11352546
Found match: https://www.sofascore.com/football/match/aston-villa-manchester-city/rP#id:11352352 with Match ID: 11352352
Found match: https://www.sofascore.com/football/match/everton-chelsea/BQy#id:11352401 with Match ID: 11352401
Found match: https://www.sofascore.com/football/match/aston-villa-crystal-palace/hP#id:11352562 with Match ID: 11352562
Found match: https://www.sofascore.com/football/match/manchester-city-crystal-palace/hr#id:11352362 with Match ID: 11352362
Found match: https://www.sofascore.com/football/match/manchester-united-sheffield-united/psK#id:12190336 with Match ID: 12190336
Found match: https://www.sofascore.com/football/match/chelsea-burn

In [9]:
#Should be 10*38=380
len(collected_match_ids)

373

In [4]:
import http.client, json
from urllib.parse import urlparse
import requests

In [4]:
#Checking the json with the data
match_id=collected_match_ids[1]

url = "https://www.sofascore.com/api/v1/event/"+match_id+"/lineups"
parsed_url = urlparse(url)
conn = http.client.HTTPSConnection(parsed_url.netloc)
conn.request("GET",parsed_url.path)
res = conn.getresponse()
data = res.read()
jsondata = json.loads(data.decode("utf-8"))
jsondata

{'confirmed': True,
 'home': {'players': [{'player': {'name': 'David Raya',
     'slug': 'david-raya',
     'shortName': 'D. Raya',
     'position': 'G',
     'jerseyNumber': '22',
     'height': 183,
     'userCount': 12362,
     'id': 581310,
     'country': {'alpha2': 'ES',
      'alpha3': 'ESP',
      'name': 'Spain',
      'slug': 'spain'},
     'marketValueCurrency': 'EUR',
     'dateOfBirthTimestamp': 811123200,
     'fieldTranslations': {'nameTranslation': {'ar': 'ديفيد مارتن رايا'},
      'shortNameTranslation': {'ar': 'د. م. رايا'}}},
    'shirtNumber': 22,
    'jerseyNumber': '22',
    'position': 'G',
    'substitute': False,
    'statistics': {'totalPass': 15,
     'accuratePass': 13,
     'totalLongBalls': 2,
     'accurateLongBalls': 1,
     'goalAssist': 0,
     'savedShotsFromInsideTheBox': 1,
     'saves': 1,
     'totalKeeperSweeper': 1,
     'accurateKeeperSweeper': 1,
     'minutesPlayed': 90,
     'touches': 21,
     'rating': 6.6,
     'possessionLostCtrl': 2,
  

In [10]:
#Decoding date of birth of player
from datetime import datetime
import time

match_id = collected_match_ids[1]

url = "https://www.sofascore.com/api/v1/event/"+match_id+"/lineups"
parsed_url = urlparse(url)
conn = http.client.HTTPSConnection(parsed_url.netloc)
conn.request("GET",parsed_url.path)
res = conn.getresponse()
data = res.read()
jsondata = json.loads(data.decode("utf-8"))

timestamp = jsondata['home']['players'][0]['player']['dateOfBirthTimestamp'] #699667200
date_object = datetime.fromtimestamp(timestamp)
print("Datetime:", date_object)

Datetime: 1995-09-14 17:00:00


In [6]:
#Just checking if all matches collected are from the correct season and tournament
for match_id in collected_match_ids:
    url = "https://www.sofascore.com/api/v1/event/"+match_id
    parsed_url = urlparse(url)
    conn = http.client.HTTPSConnection(parsed_url.netloc)
    conn.request("GET",parsed_url.path)
    res = conn.getresponse()
    data = res.read()
    jsondata = json.loads(data.decode("utf-8"))
    if jsondata['event']['season']['name'] != 'Premier League 23/24':
        print(jsondata['event']['season']['name'])

In [7]:
#Obtaining the ratings of home players
match_id = collected_match_ids[1]

url = "https://www.sofascore.com/api/v1/event/"+match_id+"/lineups"
parsed_url = urlparse(url)
conn = http.client.HTTPSConnection(parsed_url.netloc)
conn.request("GET",parsed_url.path)
res = conn.getresponse()
data = res.read()
jsondata = json.loads(data.decode("utf-8"))

for player in jsondata['home']['players']:
    player_name = player['player']['name']
    if 'rating' not in player['statistics']:
        print(player_name)
    else:
        player_rating = player['statistics']['rating']
        print(player_name,player_rating)

David Raya 6.6
Ben White 6.8
William Saliba 7.1
Gabriel Magalhães 7
Takehiro Tomiyasu 7.7
Martin Ødegaard 8.5
Thomas Partey 7
Declan Rice 7.2
Gabriel Martinelli 8
Kai Havertz 7.3
Leandro Trossard 6.7
Oleksandr Zinchenko 6.9
Jurriën Timber 6.7
Emile Smith Rowe 7.1
Gabriel Jesus 7.2
Aaron Ramsdale
Jakub Kiwior
Fábio Vieira
Jorginho
Edward Nketiah


In [14]:
import get_data.Scrapers.sofascore_scraper as sofascraper
import importlib
importlib.reload(sofascraper)

<module 'get_data.Scrapers.sofascore_scraper' from '/home/cody/Documents/DataSciBC/EPLTransfer/get_data/Scrapers/sofascore_scraper.py'>

In [16]:
data = sofascraper.get_all()
data.data.head()

Scrolled to position: 1700
Scrolled to position: 1700
Round 1: Match links located (count: 31).
11352571
Found match: https://www.sofascore.com/football/match/west-ham-united-manchester-city/rM#id:11352571 with Match ID: 11352571
11352546
Found match: https://www.sofascore.com/football/match/everton-arsenal/RY#id:11352546 with Match ID: 11352546
11352546
11352549
Found match: https://www.sofascore.com/football/match/brentford-newcastle-united/Osab#id:11352549 with Match ID: 11352549
11352552
Found match: https://www.sofascore.com/football/match/manchester-united-brighton-and-hove-albion/FsK#id:11352552 with Match ID: 11352552
11352555
Found match: https://www.sofascore.com/football/match/nottingham-forest-burnley/gso#id:11352555 with Match ID: 11352555
11352558
Found match: https://www.sofascore.com/football/match/bournemouth-chelsea/Nskb#id:11352558 with Match ID: 11352558
11352562
Found match: https://www.sofascore.com/football/match/aston-villa-crystal-palace/hP#id:11352562 with Mat

KeyboardInterrupt: 